PNN technique for data optimization and

In [ ]:
pip install tensorflow-probability

In [ ]:
pip install tensorflow-datasets


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [ ]:
def get_train_and_test_splits(train_size, batch_size=1):
    # Prefetch with a buffer the same size as the dataset because th dataset is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # Shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset


In [ ]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [ ]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs


Creating probability neural network

In [ ]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

In [ ]:
num_epochs = 10
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/10
17/17 [==============================] - 2s 65ms/step - loss: 29.4161 - root_mean_squared_error: 5.4237 - val_loss: 26.3928 - val_root_mean_squared_error: 5.1374
Epoch 2/10
17/17 [==============================] - 0s 10ms/step - loss: 27.6047 - root_mean_squared_error: 5.2540 - val_loss: 25.2274 - val_root_mean_squared_error: 5.0227
Epoch 3/10
17/17 [==============================] - 0s 11ms/step - loss: 26.0760 - root_mean_squared_error: 5.1065 - val_loss: 24.0143 - val_root_mean_squared_error: 4.9004
Epoch 4/10
17/17 [==============================] - 0s 11ms/step - loss: 24.6159 - root_mean_squared_error: 4.9614 - val_loss: 22.7766 - val_root_mean_squared_error: 4.7725
Epoch 5/10
17/17 [==============================] - 0s 11ms/step - loss: 23.1953 - root_mean_squared_error: 4.8162 - val_loss: 21.5358 - val_root_mean_squared_error: 4.6407
Epoch 6/10
17/17 [==============================] - 0s 12ms/step - loss: 21.8138 - root_mean_squared_error:

In [ ]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 1.9 - Actual: 4.0
Predicted: 2.0 - Actual: 6.0
Predicted: 2.0 - Actual: 5.0
Predicted: 2.0 - Actual: 6.0
Predicted: 2.1 - Actual: 5.0
Predicted: 2.0 - Actual: 6.0
Predicted: 2.0 - Actual: 5.0
Predicted: 2.1 - Actual: 5.0
Predicted: 2.0 - Actual: 5.0
Predicted: 1.9 - Actual: 6.0


BNN moidel

In [ ]:

def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model



def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [ ]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
num_epochs = 10
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/10
5/5 [==============================] - 3s 235ms/step - loss: 30.4242 - root_mean_squared_error: 5.5144 - val_loss: 23.6098 - val_root_mean_squared_error: 4.8574
Epoch 2/10
5/5 [==============================] - 0s 28ms/step - loss: 30.0407 - root_mean_squared_error: 5.4799 - val_loss: 30.2720 - val_root_mean_squared_error: 5.5007
Epoch 3/10
5/5 [==============================] - 0s 29ms/step - loss: 29.6040 - root_mean_squared_error: 5.4397 - val_loss: 31.0157 - val_root_mean_squared_error: 5.5679
Epoch 4/10
5/5 [==============================] - 0s 29ms/step - loss: 29.0708 - root_mean_squared_error: 5.3907 - val_loss: 30.0951 - val_root_mean_squared_error: 5.4843
Epoch 5/10
5/5 [==============================] - 0s 30ms/step - loss: 29.7677 - root_mean_squared_error: 5.4549 - val_loss: 21.6480 - val_root_mean_squared_error: 4.6513
Epoch 6/10
5/5 [==============================] - 0s 28ms/step - loss: 30.8822 - root_mean_squared_error: 5.5559 - v

In [ ]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 0.85, min: 0.03, max: 1.68, range: 1.64 - Actual: 4.0
Predictions mean: 0.84, min: 0.0, max: 1.59, range: 1.59 - Actual: 6.0
Predictions mean: 0.85, min: 0.0, max: 1.55, range: 1.55 - Actual: 5.0
Predictions mean: 0.85, min: 0.03, max: 1.53, range: 1.51 - Actual: 6.0
Predictions mean: 0.86, min: -0.0, max: 1.58, range: 1.58 - Actual: 5.0
Predictions mean: 0.84, min: 0.03, max: 1.56, range: 1.54 - Actual: 6.0
Predictions mean: 0.85, min: 0.05, max: 1.64, range: 1.59 - Actual: 5.0
Predictions mean: 0.87, min: -0.01, max: 1.58, range: 1.59 - Actual: 5.0
Predictions mean: 0.86, min: -0.0, max: 1.55, range: 1.55 - Actual: 5.0
Predictions mean: 0.84, min: 0.03, max: 1.56, range: 1.54 - Actual: 6.0


In [ ]:
num_epochs = 10
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/10
17/17 [==============================] - 3s 58ms/step - loss: 21.8791 - root_mean_squared_error: 4.6771 - val_loss: 18.4855 - val_root_mean_squared_error: 4.2989
Epoch 2/10
17/17 [==============================] - 0s 14ms/step - loss: 20.1557 - root_mean_squared_error: 4.4890 - val_loss: 23.2813 - val_root_mean_squared_error: 4.8247
Epoch 3/10
17/17 [==============================] - 0s 13ms/step - loss: 18.8150 - root_mean_squared_error: 4.3371 - val_loss: 14.6785 - val_root_mean_squared_error: 3.8307
Epoch 4/10
17/17 [==============================] - 0s 13ms/step - loss: 19.5054 - root_mean_squared_error: 4.4159 - val_loss: 15.0795 - val_root_mean_squared_error: 3.8826
Epoch 5/10
17/17 [==============================] - 0s 12ms/step - loss: 17.0479 - root_mean_squared_error: 4.1284 - val_loss: 18.1559 - val_root_mean_squared_error: 4.2604
Epoch 6/10
17/17 [==============================] - 0s 12ms/step - loss: 17.5731 - root_mean_squared_error:

PNN model and training

In [ ]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilistic output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # Set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:

def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 10
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/10
17/17 [==============================] - 4s 66ms/step - loss: 212.9756 - root_mean_squared_error: 6.4623 - val_loss: 139.8549 - val_root_mean_squared_error: 6.5212
Epoch 2/10
17/17 [==============================] - 0s 13ms/step - loss: 184.7269 - root_mean_squared_error: 6.4528 - val_loss: 220.9004 - val_root_mean_squared_error: 6.3506
Epoch 3/10
17/17 [==============================] - 0s 12ms/step - loss: 176.9464 - root_mean_squared_error: 6.3008 - val_loss: 204.8843 - val_root_mean_squared_error: 6.3638
Epoch 4/10
17/17 [==============================] - 0s 15ms/step - loss: 135.0159 - root_mean_squared_error: 6.1718 - val_loss: 85.2571 - val_root_mean_squared_error: 5.9879
Epoch 5/10
17/17 [==============================] - 0s 13ms/step - loss: 99.8310 - root_mean_squared_error: 6.1422 - val_loss: 92.1866 - val_root_mean_squared_error: 6.3537
Epoch 6/10
17/17 [==============================] - 0s 13ms/step - loss: 115.9007 - root_mean_square

In [ ]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: -0.16, stddev: 0.78, 95% CI: [1.36 - -1.68] - Actual: 4.0
Prediction mean: -0.11, stddev: 0.82, 95% CI: [1.49 - -1.71] - Actual: 6.0
Prediction mean: 0.08, stddev: 0.79, 95% CI: [1.62 - -1.46] - Actual: 5.0
Prediction mean: 0.13, stddev: 0.71, 95% CI: [1.53 - -1.27] - Actual: 6.0
Prediction mean: 0.14, stddev: 0.83, 95% CI: [1.76 - -1.49] - Actual: 5.0
Prediction mean: -0.0, stddev: 0.73, 95% CI: [1.42 - -1.42] - Actual: 6.0
Prediction mean: -0.13, stddev: 0.78, 95% CI: [1.41 - -1.67] - Actual: 5.0
Prediction mean: 0.13, stddev: 0.86, 95% CI: [1.81 - -1.56] - Actual: 5.0
Prediction mean: 0.11, stddev: 0.83, 95% CI: [1.74 - -1.52] - Actual: 5.0
Prediction mean: -0.1, stddev: 0.78, 95% CI: [1.42 - -1.63] - Actual: 6.0
